# **(Kaggle Data Collection)**

## Objectives

* Fetch data from Kaggle and save it as raw data.
* Fetch data from the SEC website and save it as raw data.
* Inspect the data and save it under data/raw/.


## Inputs

* Kaggle JSON file (kaggle.json): The authentication token for accessing Kaggle datasets.
* SEC Insider Transactions Data Sets: Downloaded either manually or via script.

## Outputs

* Datasets saved in data/raw/:
    * Kaggle Datasets:
        * financial_indicators/ containing 2014_Financial_Data.csv to 2018_Financial_Data.csv.
        * stock_prices/ containing the Huge Stock Market Dataset files.
    * SEC Insider Transactions Data Sets:
        * TSV files saved in insider_transactions/.

---

## Install Required Python Packages

First, install all required packages. You can install packages directly in the notebook using the %pip command.

In [ ]:
%pip install kaggle==1.5.12
%pip install pandas
%pip install requests
%pip install beautifulsoup4

## 